In [44]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
import scipy.stats as stats

In [94]:
#pull in dataset used as the benchmark and dataset for SMA strategy. Note the start and end dates.

start = datetime(2002,1,1)
end = datetime(2020,5,1)
stock = data.DataReader("IVV", 'yahoo', start, end).reset_index()
close = stock["Close"] #data used for SMA function

In [95]:
#The SMA function,includes current day included SMA. basic form is to compare price to SMA of length set by user.
#This function looks for another price signal after a number of days equal to the return length

def SMA_timing_func_edit(SMA_len, return_len, data, initial):
    timing = initial
    hold = 'cash'
    acct_value = []
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data[n-SMA_len:n].mean()
        acct_value.append(timing)
        if (price >= SMA) & (hold =='stock'): #the first case for when the loop checks for a price signal
            timing = next_price/price*timing
        elif (price >= SMA) & (hold =='cash'):#the second case for when the loop checks for a price signal
            hold = 'stock'
            timing = next_price/price*timing
        elif (price < SMA) & (hold =='stock'):#the third and final case for when the loop checks for a price signal because in th fourth scenario nothing happens.
            hold ='cash'
    return(timing,acct_value)

In [96]:
#calling the SMA(timing the market) function with a 14 day SMA and rechecking the price signal every day.
#initial value was chosen to be the IVV stock price at the start of the dataset for graphing purposes.
value,acct_level = SMA_timing_func_edit(14,1,close,113.650002)

In [97]:
#combined benchmark and portfolio daily values and make it loo nice
acct=pd.DataFrame(acct_level)
acct=pd.merge(acct,pd.DataFrame(stock["Date"]).iloc[15:].reset_index().drop("index",axis=1),left_index=True, right_index=True)
IVV=pd.DataFrame(close)
ports=acct.join(IVV,how="left")
ports.Date=pd.to_datetime(acct.Date)
ports.set_index("Date",inplace=True)
ports.columns=["Strat","IVV"]

In [98]:
ports.head()

,Strat,IVV
Date,,
2002-01-24,113.650002,115.449997
2002-01-25,113.650002,116.779999
2002-01-28,113.650002,117.599998
2002-01-29,113.650002,116.709999
2002-01-30,113.650002,116.230003


In [99]:
#Daily Returns
returns=ports.pct_change()

In [100]:
returns.head(10)

,Strat,IVV
Date,,
2002-01-24,NaN,NaN
2002-01-25,0.0,0.011520
2002-01-28,0.0,0.007022
2002-01-29,0.0,-0.007568
2002-01-30,0.0,-0.004113
2002-01-31,0.0,-0.005162
2002-02-01,0.0,0.004151
2002-02-04,0.0,-0.009818
2002-02-05,0.0,-0.006523


In [101]:
#drop first day NaN
returns=returns.drop(returns.index[:1])

Compare portfolio characteristics

In [102]:
#Annual returns and standard deviation and sharpe (note for sharpe ratio I simply do annual return/annual std dev so its not really the sharpe ratio)
An_port_returns = returns.resample('AS').sum()
An_port_std = returns.resample('AS').sum().std()
sharpe=An_port_returns/An_port_std

In [103]:
returns.resample('AS').sum().mean()

Strat    0.022303
IVV      0.064623
dtype: float64

In [104]:
returns.resample('AS').sum().std()

Strat    0.130112
IVV      0.167048
dtype: float64

In [105]:
sharpe.mean()

Strat    0.171417
IVV      0.386856
dtype: float64

In [106]:
#left is the SMA strategy, right is IVV
stats.skew(returns)

array([-0.33025269, -0.15480084])

In [107]:
#left is the SMA strategy, right is IVV
stats.kurtosis(returns)

array([16.39847444, 11.83676265])